In [ ]:
import boto3
from datetime import date

#Function for executing athena queries

# Using boto3 API
def run_query(query, database, s3_output): 
    client = boto3.client('athena','us-east-1') 
    response = client.start_query_execution( QueryString=query, \
                                            QueryExecutionContext={ 'Database': database }, \
                                            ResultConfiguration={ 'OutputLocation': s3_output, } )
    print('Execution ID: ' + response['QueryExecutionId']) 
    return response

#Athena configuration

def lambda_handler(event,context) :
    
    
    today = date.today()
    s3_output = 's3://athena-results/'.format(today)
    
    
    # Athena target database and table definition
    target_db = 'daily_reports_db'
    target_table = 'daily_reports_tbl'
    # S3 root where the data was saved as gz file
    target_s3_input = 's3://daily_reports/sales'
    
    create_database = "CREATE DATABASE IF NOT EXISTS {};".format(target_db)
    
    drop_table = "drop table {}.{};".format(target_db, target_table)
    create_table =  """ 
            CREATE EXTERNAL TABLE IF NOT EXISTS  {}.{} (
              `today` date,
              `date` date, 
              `customer` int, 
              `count` bigint)
            ROW FORMAT DELIMITED 
              FIELDS TERMINATED BY ',' 
            STORED AS INPUTFORMAT 
              'org.apache.hadoop.mapred.TextInputFormat' 
            OUTPUTFORMAT 
              'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
            LOCATION
              '{}'
            TBLPROPERTIES (
              'has_encrypted_data'='false');""".format(target_db, target_table, target_s3_input )
    database = 'sales_db'
    table = 'sales_customer'
    
    #Presto Query definitions
    query_1 = """
        INSERT INTO {}.{}  
        SELECT 
            current_date AS today,
            date(split_part(record_timestamp_new,' ',1))  AS date, 
            customer_id, 
            count(customer_id) AS cnt_customer
            
        FROM {}.{} 
        WHERE date(split_part(record_timestamp_new,' ',1)) 
                    BETWEEN current_date - interval '120' day
                    AND current_date
        GROUP BY  date(split_part(record_timestamp_new,' ',1)) ,customer_id
        ORDER by date,customer_id;""".format(target_db,target_table,database, table) 
    
    import time
    #Execute all queries
    queries = [ create_database, create_table, query_1 ]  
    for q in queries: 
        print("Executing query: %s" % (q))   
        res = run_query(q,target_db,s3_output)